In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statistics 
import math

In [2]:
matches = pd.read_csv("/content/sample_data/matches.csv")
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [3]:
# setting up the table with relevant columns
dropList = ["result","player_of_match","venue","umpire1","umpire2","umpire3"]
matches.drop(labels=dropList, axis=1, inplace=True)
matches.head() 

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Sunrisers Hyderabad,35,0
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,0,Rising Pune Supergiant,0,7
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,0,Kolkata Knight Riders,0,10
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,0,Kings XI Punjab,0,6
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,0,Royal Challengers Bangalore,15,0


In [4]:
matches[pd.isnull(matches['winner'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
300,301,2011,Delhi,2011-05-21,Delhi Daredevils,Pune Warriors,Delhi Daredevils,bat,0,NaN,0,0
545,546,2015,Bangalore,2015-04-29,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0
570,571,2015,Bangalore,2015-05-17,Delhi Daredevils,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,NaN,0,0
744,11340,2019,Bengaluru,30/04/19,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0


In [5]:
matches['winner'].fillna('Draw', inplace=True)
matches[pd.isnull(matches['winner'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets


In [6]:
matches[pd.isnull(matches['city'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
461,462,2014,NaN,2014-04-19,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Royal Challengers Bangalore,0,7
462,463,2014,NaN,2014-04-19,Kolkata Knight Riders,Delhi Daredevils,Kolkata Knight Riders,bat,0,Delhi Daredevils,0,4
466,467,2014,NaN,2014-04-23,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,field,0,Chennai Super Kings,7,0
468,469,2014,NaN,2014-04-25,Sunrisers Hyderabad,Delhi Daredevils,Sunrisers Hyderabad,bat,0,Sunrisers Hyderabad,4,0
469,470,2014,NaN,2014-04-25,Mumbai Indians,Chennai Super Kings,Mumbai Indians,bat,0,Chennai Super Kings,0,7
474,475,2014,NaN,2014-04-28,Royal Challengers Bangalore,Kings XI Punjab,Kings XI Punjab,field,0,Kings XI Punjab,0,5
476,477,2014,NaN,2014-04-30,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,0,Sunrisers Hyderabad,15,0


In [7]:
matches['city'].fillna('Dubai', inplace=True)
matches[pd.isnull(matches['city'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets


In [9]:
matches.replace(["Deccan Chargers","Delhi Daredevils"],["Sunrisers Hyderabad","Delhi Capitals"],inplace=True,regex=True)
match1 = matches[((matches.team1=="Royal Challengers Bangalore") & (matches.team2=="Mumbai Indians")) | ((matches.team1=="Mumbai Indians") & (matches.team2=="Royal Challengers Bangalore"))]
match1.shape[0]

25

In [10]:
mw_mi = 0 
mw_rcb = 0
lst= [i for i in match1['winner']] 
print("Win Tracker!")
for i in lst:
  if i=="Mumbai Indians":
    mw_mi += 1
  elif i=='Draw':
    continue
  else:
    mw_rcb += 1
  print(str(mw_mi)+" "+str(mw_rcb))
print("MI vs RCB : "+str(mw_mi)+" "+str(mw_rcb))

Win Tracker!
1 0
2 0
2 1
3 1
3 2
4 2
4 3
5 3
6 3
7 3
7 4
7 5
8 5
8 6
9 6
9 7
10 7
11 7
11 8
12 8
13 8
14 8
14 9
15 9
16 9
MI vs RCB : 16 9


In [11]:
last_3_season = matches[(matches['season']>=2017) & np.logical_or(np.logical_and(matches.team1=="Mumbai Indians",matches.team2=="Royal Cahllengers Bangalore"),np.logical_and(matches.team1=="Royal Challengers Bangalore",matches.team2=="Mumbai Indians"))]
last_3_season.groupby('winner').season.count()

winner
Mumbai Indians                 3
Royal Challengers Bangalore    1
Name: season, dtype: int64

Out of 26 matches held between MI and RCB, MI leads against RCB with 16 is to 9.Same is the case with last three seasons, MI has got clear lead of victories over RCB i.e 3 is to 1

In [49]:
deliveries = pd.read_csv("/content/sample_data/deliveries.csv")
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN


In [50]:
dropToBeList = ['inning','is_super_over','bye_runs','legbye_runs','fielder']
deliveries.drop(dropToBeList, axis=1, inplace=True)
deliveries.replace(['Deccan Chargers','Delhi Daredevils'],['Sunrisers Hyderabad','Delhi Capitals'],inplace=True,regex=True)
deliveries['dismissal_kind'].fillna('Not Out',inplace=True)
deliveries.head()

,match_id,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,wide_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind
0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
2,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,4,0,4,NaN,Not Out
3,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
4,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,2,0,0,0,2,2,NaN,Not Out


In [51]:
ballbyball = deliveries[((deliveries.batting_team=="Mumbai Indians") & (deliveries.bowling_team=="Royal Challengers Bangalore")) | ((deliveries.batting_team=="Royal Challengers Bangalore") & (deliveries.bowling_team=="Mumbai Indians"))]
no_of_matches=list(set([i for i in ballbyball['match_id']]))
no_of_matches.sort()
print(len(no_of_matches))

25


In [52]:
#Q2
array=[]
for i in no_of_matches:
  df = ballbyball[ballbyball['match_id']==i]
  bowlers = list(set([i for i in df['bowler']]))
  count=0
  for j in bowlers:
    bowler_deli = df[(df['bowler']==j) & (df['dismissal_kind'].isin(['caught','bowled','caught and bowled','lbw']))]
    no_wkts = bowler_deli.dismissal_kind.count()
    if(no_wkts>=2):
      count += 1
  array.append(count)
print(array)
print(statistics.median(array))
print(statistics.mean(array))
print(statistics.median_high(array))

[2, 3, 2, 2, 1, 0, 2, 4, 3, 2, 3, 3, 0, 2, 2, 4, 3, 3, 1, 2, 1, 5, 4, 4, 2]
2
2.4
2


There might be maximum of 5 bowlers and on an average of 2 to 3 bowlers who gets 2 wickets in their quota!

In [53]:
#Q3
arr = []
for i in no_of_matches:
  df = ballbyball[ballbyball['match_id']==i]
  bowlers = list(set([i for i in df['bowler']]))
  count=0
  for j in bowlers:
    bowler_deli = df[(df['bowler']==j)]
    runs_given = bowler_deli.total_runs.sum()
    nof_overs = bowler_deli.over.nunique()
    if(nof_overs>1 and (round((runs_given/nof_overs),2)<8)):
      count +=1
  arr.append(count)
print(arr)
print(statistics.mean(arr))
print(statistics.median(arr))
print(statistics.median_high(arr))


[7, 3, 6, 6, 6, 7, 5, 6, 4, 7, 5, 6, 6, 6, 3, 7, 2, 2, 2, 3, 6, 3, 4, 4, 5]
4.84
5
5


There might be chance of maximum of 7 bowlers having economy of less than 8.On an average, there may be 4 to 5 bowlers having economy less than 8

In [66]:
#Q4
ary=[]
for i in no_of_matches:
  df = ballbyball[ballbyball['match_id']==i]
  batter=[]
  for i in df['batsman']:
    if i not in batter:
      batter.append(i)
  count = 0
  for j in batter:
    batsman_deli = df[df['batsman']==j]
    runs_scored = batsman_deli.batsman_runs.sum()
    nof_nob_wide = batsman_deli[(batsman_deli.noball_runs!=0) | (batsman_deli.wide_runs!=0)]
    not_count_balls = nof_nob_wide.shape[0]
    balls_faced = batsman_deli.shape[0]
    valid = balls_faced - not_count_balls
    strike_rate = round((runs_scored*100/valid),2)
    boundaries = batsman_deli[batsman_deli['batsman_runs']==4]
    nof_boundaries = boundaries.shape[0]
    if(balls_faced>10 and strike_rate>130 and nof_boundaries>3):
      count += 1
  ary.append(count)
print(ary)
print(statistics.mean(ary))
print(statistics.median(ary))
print(max(ary))

[1, 2, 2, 1, 2, 0, 0, 1, 0, 0, 3, 1, 3, 2, 2, 0, 2, 5, 3, 3, 0, 3, 2, 5, 4]
1.88
2
5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in long_scalars


From both sides, there are on an average 2-5 players in a match, who have faced > 10 balls,strike rate > 130 and boundaries smashed > 3

In [65]:
#Q5
mi_pp = ballbyball[(ballbyball.batting_team=="Mumbai Indians") & (ballbyball.over>=1) & (ballbyball.over<=5)].groupby('match_id').total_runs.sum()
lst1=[i for i in mi_pp]
rcb_pp = ballbyball[(ballbyball.batting_team=="Royal Challengers Bangalore") & (ballbyball.over>=1) & (ballbyball.over<=5)].groupby('match_id').total_runs.sum()
lst2=[i for i in rcb_pp]
diff=[]
for i in range(len(lst2)):
  diff.append(abs(lst2[i]-lst1[i]))
print(diff)
print(statistics.mean(diff))
print(statistics.median(diff))
print(statistics.median_high(diff))


[16, 7, 4, 2, 28, 11, 3, 1, 2, 13, 26, 15, 6, 1, 4, 7, 12, 19, 6, 2, 11, 21, 4, 5, 12]
9.52
7
7


Maximum of 26 runs and on an average of 7 to 10 runs difference MI and RCB after their respective powerplay 